<a href="https://colab.research.google.com/github/oldflag/Symptom_Dx_ICD/blob/main/Symptom_Dx_ICD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import userdata
HF_KEY = userdata.get('HF_TOKEN')
TRANSFORMERS_CACHE = userdata.get('TRANSFORMERS_CACHE')
HF_HOME = userdata.get('HF_HOME')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
!pip install langchain openai huggingface_hub langchain_community langchain_openai faiss-gpu

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="oldflag/symptom_dx_finetue_Llama-3_8b_Unsloth",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 128},
)

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "oldflag/symptom_dx_finetue_Llama-3_8b_Unsloth"
tokenizer = AutoTokenizer.from_pretrained(
    model_id
)
model = AutoModelForCausalLM.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model,
                tokenizer=tokenizer, max_new_tokens=512)

hf = HuggingFacePipeline(pipeline=pipe)

In [6]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Based on the following symptoms, suggest a possible diagnosis

### Input:
{input}

### Response:

"""
prompt = PromptTemplate.from_template(template)

# chain
chain = prompt | hf | StrOutputParser()



In [7]:
symptom = "I have a skin rash that gets worse in the winter when the air is dry. I have to moisturize more regularly and use humidifiers to keep my skin moisturized. I also have joint pain"


print(
    chain.invoke({"input": symptom})
)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Based on the following symptoms, suggest a possible diagnosis

### Input:
I have a skin rash that gets worse in the winter when the air is dry. I have to moisturize more regularly and use humidifiers to keep my skin moisturized. I also have joint pain

### Response:

psoriasis


In [26]:
import sys
dir = """/content/drive/MyDrive/Colab Notebooks/LangChains/"""
sys.path.append(dir)
from rag_icdcoding import *


In [36]:
icdfile_path = '/content/drive/MyDrive/Colab Notebooks/LangChains/icd10cm_order_2024_top100.txt'  # only first 100 rows for testing
icd10_df = parse_icd10_description(icdfile_path)
data = load_dfdata(icd10_df)
vectordb = create_vectorstore(data)
# Initialize the language model (LLM)
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
chain_demo = extract_demo(llm, vectordb)
chain_icd = extract_icd(llm, vectordb)


In [37]:
note1 = """
Patient Name: John Doe1
Date of Birth: 01/15/1980
Visit Date: 06/11/2024
Provider: Dr. Jane Smith

Chief Complaint:
Fever, abdominal pain, and diarrhea for five days.

History of Present Illness:
John Doe, 44, reports high fever, severe abdominal cramps, and persistent diarrhea. Symptoms began five days ago and have worsened. Denies recent travel but consumed undercooked poultry a week ago.

Physical Examination:

    • Temperature: 102.2°F
    • Abdomen: Tenderness in the right lower quadrant with guarding.

Assessment and Plan:

    1. typhoid fever.
    • Order blood cultures and stool tests.
    • Start empiric antibiotic therapy with ceftriaxone.
    2. Salmonella enteritis.
    • Maintain hydration with oral rehydration solutions.
    • Educate on safe food handling and proper cooking techniques.

Follow-Up:
Return in 48 hours for follow-up and test results review. Immediate return if symptoms worsen.

Signature:
Dr. Jane Smith, MD

"""
# assessment = create_chain_2(llm, note1)
# print(assessment)
assessment = extract_assessment(llm, note1)
print(chain_demo.invoke(note1))
print(chain_icd.invoke(assessment))

Patient Name: John Doe
Age: 44
Based on the provided note, the related ICD codes are as follows:

1. Typhoid fever: ICD-10 Code A010
2. Salmonella enteritis: ICD-10 Code A020


In [38]:
chain_icd.invoke("psoriasis")

'Based on the provided note, the related ICD codes for psoriasis would be:\n- A074 for Cyclosporiasis\n- A073 for Isosporiasis\n- A067 for Cutaneous amebiasis\n\nThese are the closest matches based on the information provided.'